Importing the Dependencies

In [ ]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection - Kaggle API

In [ ]:
kaggle_dictionary = json.load(open('kaggle.json'))

In [ ]:
# setup kaggle credentials as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 599MB/s]


In [ ]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [ ]:
# unzip the dataset file
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
  zip_ref.extractall()

In [ ]:
!ls # to see the extracted file

'IMDB Dataset.csv'			 kaggle.json
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


Loading the Dataset

In [ ]:
data = pd.read_csv('IMDB Dataset.csv')

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.tail() # for printing the last five rows

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [ ]:
# How is the data is distributed?
data['sentiment'].value_counts()

# there is no class imbalance

,count
sentiment,
positive,25000
negative,25000


In [ ]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)

<ipython-input-16-2164277587>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)


In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [ ]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Pre-Processing

In [ ]:
# tokenize text data
tokenizer = Tokenizer(num_words=5000) # coverts words/phrases into numbers (we're going to take the most common 5000 words in this data)
tokenizer.fit_on_texts(train_data['review'])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen = 200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen = 200)

In [ ]:
print(x_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [ ]:
y_trian = train_data['sentiment']
y_test = test_data['sentiment']

LSTM - Long Short Term Memory

In [ ]:
# build the model

import numpy as np

model = Sequential()

model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))  # 5000 is the number of words or the vocabulary,this layer represents the data in it's vector form(128)
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = 'sigmoid'))

model(np.zeros((1, 200)).astype(np.int32))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.50422555]], dtype=float32)>

In [ ]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (1, 200, 128)          │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (1, 128)               │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (1, 1)                 │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# compile model

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# trainig the model
model.fit(x_train, y_trian, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 392s 775ms/step - accuracy: 0.7301 - loss: 0.5255 - val_accuracy: 0.8489 - val_loss: 0.3591
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 359s 718ms/step - accuracy: 0.8476 - loss: 0.3649 - val_accuracy: 0.8400 - val_loss: 0.3602
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 363s 680ms/step - accuracy: 0.8711 - loss: 0.3152 - val_accuracy: 0.8676 - val_loss: 0.3179
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 383s 681ms/step - accuracy: 0.9025 - loss: 0.2486 - val_accuracy: 0.8756 - val_loss: 0.3122
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 380s 678ms/step - accuracy: 0.9143 - loss: 0.2213 - val_accuracy: 0.8579 - val_loss: 0.3444


Model Evaluation

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 91ms/step - accuracy: 0.8647 - loss: 0.3323
Test loss: 0.32671546936035156
Test accuracy: 0.8683000206947327


Building Predictive System

In [ ]:
def predict_sentiment(review):
  # tokenize and pad the review text
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen = 200)

  prediction = model.predict(padded_sequence)

  sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'

  return sentiment

In [ ]:
# example usage

new_review = "The movie was not only good, but the acting was also amazing!"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
The sentiment of the review is: positive
